# Speech-to-Text

# Loading the Dataset and Transcription File

In [1]:
# Importing the libraries
import os
import pandas as pd
import re
import librosa
from datasets import Dataset

In [2]:
# loading the data and the transcription file
data_directory = "../Badaga_Corpus-v.0.1.0/"
tagged_file = "Badaga-v0.1.0.xlsx"

In [3]:
# joining the path to the root directory
tagged_file_path = os.path.join(data_directory, tagged_file)

# reading the data
data_frame = pd.read_excel(tagged_file_path)

# dropping missing values
data_frame.dropna(inplace=True)

# reading the audio files
data_frame["audio_file_name"] = data_frame["audio_file_name"].apply(lambda x: os.path.join(data_directory, "clips", x))

# splitting the data based on train and test in the transcription file
train_df = data_frame[data_frame["split_label"]!="test"]
test_df = data_frame[data_frame["split_label"]=="test"]

In [4]:
# printing the shape of the train and test
train_df.shape, test_df.shape

((8365, 9), (1469, 9))

# Preprocessing the Data

In [ ]:
# loading the audio using librosa 
audio = list()
for file in list(train_df["audio_file_name"]):
    a, s = librosa.load(file, sr=16000)
    audio.append({
        'path': file,
        'array': a,
        'sampling_rate': s
    })
    




audio1 = list()
for file in list(test_df["audio_file_name"]):
    a, s = librosa.load(file, sr=16000)
    audio1.append({
        'path': file,
        'array': a,
        'sampling_rate': s
    })
    



In [6]:
# creating the dictionary for both train and test by taking the audiofile and corresponding translated sentences
train_dict = {'client_id':list(range(0, len(audio))), 'audio': audio, "sentence": list(train_df["translterated_script"])}
test_dict = {'client_id':list(range(len(audio), len(audio)+len(audio1))),'audio': audio1, "sentence": list(test_df["translterated_script"])}

In [7]:
# creating a dataset for the above created dictionary
train_dataset = Dataset.from_dict(train_dict)
test_dataset = Dataset.from_dict(test_dict)

In [8]:
# printing the column names
train_dataset.column_names

['client_id', 'audio', 'sentence']

In [ ]:
# printing the first column using index
train_dataset[0]

In [10]:
# function to remove the special characters
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�]'


def remove_special_characters(batch):
    batch["sentence"] = re.sub(chars_to_ignore_regex, '', batch["sentence"]).upper() + " "
    return batch

In [11]:
# removing the special characters from train and test using map function
train_dataset = train_dataset.map(remove_special_characters)
test_dataset = test_dataset.map(remove_special_characters)

  0%|          | 0/8365 [00:00<?, ?ex/s]

  0%|          | 0/1469 [00:00<?, ?ex/s]

In [ ]:
# checking the dataset after removing special characters
train_dataset[0]

# Tokenizing and Feature Extraction

In [13]:
# loading the tokenizer for tokenizing
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer("../configs/stt-vocab.json", unk_token="<unk>", pad_token="<pad>", word_delimiter_token="|")

In [14]:
# loading the feature extractor for feature extraction
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)

In [15]:
# loading the processor which takes the tokenizer and feature extractor to process
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [ ]:
# printing the data after processing 
train_dataset[0]

In [ ]:
train_dataset[0]

In [18]:
# function to pass the audio files in batch
def prepare_dataset(batch):
    audio = batch["audio"]

    # batched output is "un-batched"
    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    
    with processor.as_target_processor():
        batch["labels"] = processor(batch["sentence"]).input_ids
    return batch

In [19]:
# map function to map the train and test dataset
train_dataset = train_dataset.map(prepare_dataset, remove_columns=train_dataset.column_names)
test_dataset = test_dataset.map(prepare_dataset, remove_columns=test_dataset.column_names)

  0%|          | 0/8365 [00:00<?, ?ex/s]

  0%|          | 0/1469 [00:00<?, ?ex/s]

# Fine-Tunning the Wav2Vec Model

In [20]:
# defining the function for data collator
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [21]:
# data collator
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [22]:
# setting the Word Error Rate(WER) metric
from datasets import load_metric
wer_metric = load_metric("wer")

In [23]:
# function for computing the WER 
import numpy as np
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [24]:
# loading the XLSR-53 pretrained model
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-large-xlsr-53", 
    attention_dropout=0.1,
    hidden_dropout=0.1,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.1,
    ctc_loss_reduction="mean", 
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer)
)

Some weights of the model checkpoint at facebook/wav2vec2-large-xlsr-53 were not used when initializing Wav2Vec2ForCTC: ['project_q.bias', 'project_q.weight', 'quantizer.weight_proj.bias', 'project_hid.weight', 'project_hid.bias', 'quantizer.weight_proj.weight', 'quantizer.codevectors']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-xlsr-53 and are newly initialized: ['lm_head.weight', 'lm_head.bias']
You should probably TRAIN this model on a down-stream task to be able to u

In [25]:
model.freeze_feature_extractor()
model.gradient_checkpointing_enable()

In [26]:
# setting up the arguments for training
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir="wav2vec2-rbg-badaga-stt",
  group_by_length=True,
  per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
  gradient_accumulation_steps=2,
  evaluation_strategy="steps",
  num_train_epochs=10,
  fp16=True,
  save_steps=100,
  eval_steps=100,
  logging_steps=10,
  learning_rate=3e-4,
  warmup_steps=500,
  save_total_limit=2,
)

In [27]:
# setting up the trainer
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=processor.feature_extractor,
)

Using amp half precision backend


In [28]:
# training 
trainer.train()

***** Running training *****
  Num examples = 8365
  Num Epochs = 10
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 2
  Total optimization steps = 10460
/home/ubuntu/environments/corepool/lib/python3.8/site-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:1055: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  return (input_length - kernel_size) // stride + 1


Step,Training Loss,Validation Loss,Wer
100,3.605200,3.340568,1.000000
200,2.963900,2.820786,1.000000
300,2.815200,2.814020,1.000000
400,2.799100,2.767837,1.000000
500,2.710800,2.603930,1.000000
600,2.066700,1.772958,1.059333
700,1.545300,1.190826,0.913440
800,1.325100,0.956923,0.786434
900,1.268000,0.865949,0.697041
1000,1.137000,0.729941,0.597734


***** Running Evaluation *****
  Num examples = 1469
  Batch size = 4
Saving model checkpoint to wav2vec2-rbg-badaga-stt/checkpoint-100
Configuration saved in wav2vec2-rbg-badaga-stt/checkpoint-100/config.json
Model weights saved in wav2vec2-rbg-badaga-stt/checkpoint-100/pytorch_model.bin
Configuration saved in wav2vec2-rbg-badaga-stt/checkpoint-100/preprocessor_config.json
/home/ubuntu/environments/corepool/lib/python3.8/site-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:1055: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  return (input_length - kernel_size) // stride + 1
***** Running Evaluation *****
  Num examples = 1469
  Batch size = 4
Saving

Configuration saved in wav2vec2-rbg-badaga-stt/checkpoint-900/preprocessor_config.json
Deleting older checkpoint [wav2vec2-rbg-badaga-stt/checkpoint-700] due to args.save_total_limit
/home/ubuntu/environments/corepool/lib/python3.8/site-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:1055: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  return (input_length - kernel_size) // stride + 1
***** Running Evaluation *****
  Num examples = 1469
  Batch size = 4
Saving model checkpoint to wav2vec2-rbg-badaga-stt/checkpoint-1000
Configuration saved in wav2vec2-rbg-badaga-stt/checkpoint-1000/config.json
Model weights saved in wav2vec2-rbg-badaga-stt/checkpoint-

Configuration saved in wav2vec2-rbg-badaga-stt/checkpoint-1700/preprocessor_config.json
Deleting older checkpoint [wav2vec2-rbg-badaga-stt/checkpoint-1500] due to args.save_total_limit
/home/ubuntu/environments/corepool/lib/python3.8/site-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:1055: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  return (input_length - kernel_size) // stride + 1
***** Running Evaluation *****
  Num examples = 1469
  Batch size = 4
Saving model checkpoint to wav2vec2-rbg-badaga-stt/checkpoint-1800
Configuration saved in wav2vec2-rbg-badaga-stt/checkpoint-1800/config.json
Model weights saved in wav2vec2-rbg-badaga-stt/checkpoin

...

Configuration saved in wav2vec2-rbg-badaga-stt/checkpoint-7300/preprocessor_config.json
Deleting older checkpoint [wav2vec2-rbg-badaga-stt/checkpoint-7100] due to args.save_total_limit
/home/ubuntu/environments/corepool/lib/python3.8/site-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:1055: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  return (input_length - kernel_size) // stride + 1
***** Running Evaluation *****
  Num examples = 1469
  Batch size = 4
Saving model checkpoint to wav2vec2-rbg-badaga-stt/checkpoint-7400
Configuration saved in wav2vec2-rbg-badaga-stt/checkpoint-7400/config.json
Model weights saved in wav2vec2-rbg-badaga-stt/checkpoin

Configuration saved in wav2vec2-rbg-badaga-stt/checkpoint-8100/preprocessor_config.json
Deleting older checkpoint [wav2vec2-rbg-badaga-stt/checkpoint-7900] due to args.save_total_limit
/home/ubuntu/environments/corepool/lib/python3.8/site-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:1055: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  return (input_length - kernel_size) // stride + 1
***** Running Evaluation *****
  Num examples = 1469
  Batch size = 4
Saving model checkpoint to wav2vec2-rbg-badaga-stt/checkpoint-8200
Configuration saved in wav2vec2-rbg-badaga-stt/checkpoint-8200/config.json
Model weights saved in wav2vec2-rbg-badaga-stt/checkpoin

Configuration saved in wav2vec2-rbg-badaga-stt/checkpoint-8900/preprocessor_config.json
Deleting older checkpoint [wav2vec2-rbg-badaga-stt/checkpoint-8700] due to args.save_total_limit
/home/ubuntu/environments/corepool/lib/python3.8/site-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:1055: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  return (input_length - kernel_size) // stride + 1
***** Running Evaluation *****
  Num examples = 1469
  Batch size = 4
Saving model checkpoint to wav2vec2-rbg-badaga-stt/checkpoint-9000
Configuration saved in wav2vec2-rbg-badaga-stt/checkpoint-9000/config.json
Model weights saved in wav2vec2-rbg-badaga-stt/checkpoin

Configuration saved in wav2vec2-rbg-badaga-stt/checkpoint-9700/preprocessor_config.json
Deleting older checkpoint [wav2vec2-rbg-badaga-stt/checkpoint-9500] due to args.save_total_limit
/home/ubuntu/environments/corepool/lib/python3.8/site-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:1055: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  return (input_length - kernel_size) // stride + 1
***** Running Evaluation *****
  Num examples = 1469
  Batch size = 4
Saving model checkpoint to wav2vec2-rbg-badaga-stt/checkpoint-9800
Configuration saved in wav2vec2-rbg-badaga-stt/checkpoint-9800/config.json
Model weights saved in wav2vec2-rbg-badaga-stt/checkpoin

TrainOutput(global_step=10460, training_loss=0.6890156001821303, metrics={'train_runtime': 25906.7641, 'train_samples_per_second': 3.229, 'train_steps_per_second': 0.404, 'total_flos': 6.123033781781698e+18, 'train_loss': 0.6890156001821303, 'epoch': 10.0})